# [K3D Jupyter](https://github.com/K3D-tools/K3D-jupyter)


설치 방법 

```
$ pip install k3d
$ jupyter nbextension install --py --sys-prefix k3d
$ jupyter nbextension enable --py --sys-prefix k3d
```

## 1. PCD파일 읽어 표현하기

In [48]:
import pcl
import k3d
import numpy as np

In [39]:
pc = pcl.load("sample_sec.pcd")
pa = pc.to_array()

In [55]:
plot = k3d.plot()

points_number = pa.shape[0]
colors = numpy.random.randint(0, 0xFFFFFF, points_number)

points = k3d.points(pa, colors, point_size=0.01, shader='3d')
plot += points
plot.camera_auto_fit = False
plot.display()

Output()

## 2. 이미지로 저장하기 (Screen shot)

In [50]:
plot.fetch_screenshot()

In [51]:
from IPython.display import Image
with open('screenshot.png', 'wb') as f:
    try:
        out = plot.screenshot.decode('base64')
    except: # Python 3
        from base64 import b64decode
        out = b64decode(plot.screenshot)    
    f.write(out)
    
Image(url='screenshot.png')

---

## [참고] k3d의 추가 기능

## Text 입력하기

In [57]:
import k3d

plot = k3d.plot()
plot += k3d.text('\\int_{-\\infty}^\\infty \\hat f(\\xi)\\,e^{2 \\pi i \\xi x} \\,d\\xi', [0,0,0], color=0, size=1.5, reference_point='lt')
plot += k3d.text('{(1,1,\\frac{5}{\\pi})}', [0,0,0], color=0, size=1.5, reference_point='rb')
plot.display()

Output()

In [59]:
import k3d

plot = k3d.plot()
plot += k3d.text2d('\\int_{-\\infty}^\\infty \\hat f(\\xi)\\,e^{2 \\pi i \\xi x} \\,d\\xi', [0.75,0.5], color=0, size=1.5, reference_point='lt')
plot += k3d.text2d('{(1,1,\\frac{5}{\\pi})}', [0.25, 0.5], color=0, size=1.5, reference_point='rb')
plot.display()

Output()

## 에니메이션 효과

In [65]:
import k3d
import numpy as np
import time 

plot = k3d.plot()

Nx,Ny = 10,10

x = np.linspace(-1,1,Nx)
y = np.linspace(-2,2,Ny)

x,y = np.meshgrid(x,y)
z = np.zeros_like(x)

origins = np.vstack([x.flatten(),y.flatten(),z.flatten()]).T
vectors = 0.5*(np.vstack([np.zeros_like(x.flatten()),np.zeros_like(x.flatten()),np.ones_like(x.flatten())] )).T
phi0 = np.linspace(0,2*np.pi,vectors.shape[0])
labels = [str(i) for i in range(vectors.shape[0])]
vector_plot = k3d.vectors(origins, vectors)

plot += vector_plot
plot.camera_auto_fit = False
plot.grid_auto_fit = False
plot.display()

Output()

In [66]:
for phi in np.linspace(0,2*np.pi,55):
    r = 0.1  
    v = vector_plot.vectors.copy()
    v[:,0] += r*np.cos(phi+phi0)
    v[:,1] += r*np.sin(phi+phi0)
    vector_plot.vectors = v
    time.sleep(0.05) # have to wait for real data update

## voxel_callback

In [12]:
import k3d
import numpy as np
from math import sqrt, sin, cos
from skimage.measure import label

width = height = length = 100

def r(x, y, z):
    r = sqrt((x - width / 2) * (x - width / 2) + (y - height / 2) * (y - height / 2) + (z - length / 2) * (z - length / 2))
    r += sin(x / 2) * 3
    r += cos(y / 10) * 5
    
    return r

def f(x, y, z):
    return 0 if r(x, y, z) > width / 2 else (1 if y + sin(x / 20) * 10 > height / 2 else 2)

def on_click(x, y, z):
    coords = [x, y, z]
    missing = None
    missing_count = 0
    
    for idx, v in enumerate([np.array([0,0,1]), np.array([0,1,0]), np.array([1,0,0])]):
        axis = coords[idx]        
        
        if axis == 0 or (obj.voxels[tuple(np.array([z, y, x]) - v)] == 0):
            missing = -v
            missing_count += 1

        if axis == obj.voxels.shape[2 - idx] - 1 or obj.voxels[tuple(np.array([z, y, x]) + v)] == 0:
            missing = v
            missing_count += 1 
    
    if missing_count == 1:        
        if missing[0] != 0:
            slice = obj.voxels[z, :, :]
            mask = label(slice, connectivity = 2)            
            slice[mask == mask[y, x]] = plot.voxel_paint_color
        if missing[1] != 0:
            slice = obj.voxels[:, y, :]
            mask = label(slice, connectivity = 2)
            slice[mask == mask[z, x]] = plot.voxel_paint_color
        if missing[2] != 0:
            slice = obj.voxels[:, :, x]
            mask = label(slice, connectivity = 2)
            slice[mask == mask[z, y]] = plot.voxel_paint_color
            
        obj.push_data('voxels')
        
color_map = (0xffff00, 0xff0000, 0x00ff00)
voxels = np.array([[[f(x, y, z) for x in range(width)] for y in range(height)] for z in range(length)])

obj = k3d.voxels(voxels, color_map)
plot = k3d.plot()
plot += obj

obj.click_callback = on_click
plot.voxel_paint_color = 3
plot.camera_auto_fit = False

ImportError: No module named skimage.measure

In [ ]:
# select Controls->Mode->Callback and click on object wall
plot.display()